<a href="https://colab.research.google.com/github/ozturkgizem/Agriculture/blob/main/feng%26dp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [23]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
df = pd.read_csv("Crop_recommendation.csv")

**FEATURE ENG & DATA PROCESSING**

In [6]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    # print(f"Observations: {dataframe.shape[0]}")
    # print(f"Variables: {dataframe.shape[1]}")
    # print(f'cat_cols: {len(cat_cols)}')
    # print(f'num_cols: {len(num_cols)}')
    # print(f'cat_but_car: {len(cat_but_car)}')
    # print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

In [25]:
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [7]:
def crop_recommendation_data_prep(dataframe):

    ##1: CROP SUITABILITY INDICES

    cat_cols, num_cols, cat_but_car = grab_col_names(dataframe)

    def crop_suitability_indices(crop, col):
        low_limit, up_limit = outlier_thresholds(dataframe[dataframe["label"] == crop], col)
        dataframe.loc[(df["label"] == crop) & (dataframe[col] >= low_limit) & (dataframe[col] <= up_limit), f"NEW_{col.upper()}_OPTIMAL"] = 1
        dataframe.loc[(df["label"] == crop) & (dataframe[col] < low_limit) | (dataframe[col] > up_limit), f"NEW_{col.upper()}_OPTIMAL"] = 0

    for crop in df["label"].unique():
        for col in num_cols:
            crop_suitability_indices(crop, col)

    ##2: NUTRIENT BALANCE INDEX (NBI)

    dataframe["N/K"]= dataframe["N"] / dataframe["K"]
    dataframe["N/P"] = dataframe["N"] / dataframe["P"]
    dataframe["P/K"] = dataframe["P"] / dataframe["K"]

    ratios = ["N/K","N/P","P/K"]
    dataframe[ratios] = MinMaxScaler().fit_transform(dataframe[ratios])

    dataframe["NEW_NBI"] = np.sqrt(dataframe["N/K"] * dataframe["N/P"] * dataframe["P/K"])
    dataframe.drop(ratios, axis=1, inplace=True)


    ##3: CATEGORIZE BY PH LEVELS

    dataframe.loc[dataframe["ph"] < 7, "NEW_PH_CAT"] = "acidic"
    dataframe.loc[dataframe["ph"] > 7, "NEW_PH_CAT"] = "alkaline"
    dataframe.loc[dataframe["ph"] == 7, "NEW_PH_CAT"] = "neutral"
    dataframe.loc[(dataframe["ph"] >= 5.5) & (dataframe["ph"] <= 6.5), "NEW_PH_CAT"] = "optimal"

    ##4: CATEGORIZE BY RAINFALL
    #sns.catplot(data=df, x='label', y="rainfall", kind='box', height=10, aspect=22/8)
    #plt.title(f"{col}", size=12)
    #plt.xticks(rotation='vertical')
    #plt.show()

    #df["rainfall"].describe([0.10,0.20,0.30,0.33,0.40,0.50,0.60,0.66,0.70,0.75,0.80,0.90,0.99])

    dataframe["NEW_RAINFALL_CAT"] = pd.cut(x=dataframe["rainfall"], bins=[0, 44, 71, 111, 188, 300], labels=["extreme_low", "low", "medium", "high", "extreme_high"])

    ##5. ONE HOT ENCODER
    def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
        dff = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first, dtype=int)
        return dff

    cat_cols, num_cols, cat_but_car = grab_col_names(dataframe)

    dff = one_hot_encoder(dataframe, cat_cols, drop_first=True)

    cat_cols, num_cols, cat_but_car = grab_col_names(dff)

    X_scaled = StandardScaler().fit_transform(dff[num_cols])
    dff[num_cols] = pd.DataFrame(X_scaled, columns=dff[num_cols].columns)

    y = LabelEncoder().fit_transform(dff["label"])
    X = dff.drop(["label"], axis=1)

    return X, y

In [12]:
X, y = crop_recommendation_data_prep(df)

In [16]:
def base_models(X, y, scoring="roc_auc"):
    print("Base Models....")
    classifiers = [('LR', LogisticRegression()),
                   ('KNN', KNeighborsClassifier()),
                   ("SVC", SVC()),
                   ("CART", DecisionTreeClassifier()),
                   ("RF", RandomForestClassifier()),
                   ('Adaboost', AdaBoostClassifier()),
                   ('GBM', GradientBoostingClassifier()),
                   ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
                   ('LightGBM', LGBMClassifier()),
                   # ('CatBoost', CatBoostClassifier(verbose=False))
                   ]
    for name, classifier in classifiers:
        cv_results = cross_validate(classifier, X, y, cv=3, scoring=scoring)
        print(f"{scoring}: {round(cv_results['test_score'].mean(), 4)} ({name}) ")

In [ ]:
base_models(X,y)